In [2]:
from LSTM import LSTM
import torch
import numpy as np 
from torch import nn

In [ ]:
################
#  DATALOADING #
################

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)


In [ ]:
loss_fn = torch.nn.MSELoss(size_average=False)
 
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
 
#####################
# Train model
#####################
 
hist = np.zeros(num_epochs)
 
for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)
 
    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
 
    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()
 
    # Backward pass
    loss.backward()
 
    # Update parameters
    optimiser.step()